# nbdocker: quick demo
 
### By: Jiaming Hu, Ling-Hong Hung and Ka Yee Yeung. 

Institute of Technology, University of Washington Tacoma, WA 98402, USA

This is a quick demo of nbdocker, showing the feasibility of embedding a Docker container inside a Jupyter notebook.

## Introduction

In this quick demo, we will download fastq files from NCBI Short Read Archive (SRA) and then perform quality assessment using a R/Bioconductor package called ShortRead.  

**This demo shows the following:**
1. Docker containers can be embedded inside Jupyter notebooks using nbdocker. In addition, the history of the embedded containers are included in the notebook.  Sharing the notebook (.ipynb) files will include the history of Docker commands.
2. The users can  can run and monitor the progress of the embedded Docker containers by a single click.
3. We can mix and match computing environments and programming langugages. In particular, the download step involves shell commands and Python wrapper, while the quality control step uses R.
4. Allowing multiple computing environment facilitates collaboration and reproducibility.

**To run this demo nbdocker notebook:**
1. Pull the nbdocker image from DockerHub: 
>docker pull biodepot/nbdocker
2. Run the nbdocker image. The following command works on the mac.
>docker run -it -p 8888:8888 -v /var/run/docker.sock:/var/run/docker.sock -v /Users/username/nbdocker:/home/jovyan/work --privileged --group-add root biodepot/nbdocker

The above command maps the local directory */Users/username/nbdocker* on the host to */home/jovyan/work* inside the container running nbdocker.  Replace */Users/username/nbdocker* with the local directory that you'd like to map to your notebook.  Note that */home/jovyan/work* is equivalent to */.nbdocker* within the nbdocker notebook.


## 1. Download the fastq input files from SRA 
#### Note: running this container takes 5-20 minutes depending on your bandwidth.

We use RNA-seq data generated by Trapnell et al. (Nat Biotechnol. 2013 Jan; 31(1): 10.1038/nbt.2450) in which they studied the differential gene expression of lung fibroblasts in response to loss of the developmental transcription factor HOXA1.  In this case study, we will compare changes in gene expression  in response to HOXA1 knockdown. 

The fastq files are publicly available from the Short Read Archive (SRA) with the following link:
https://trace.ncbi.nlm.nih.gov/Traces/sra/?run=SRR493366

To download the files we use the parallel-fastq-dump container. This is a python wrapper around the fastq-dump utility that allows it to download separate chunks of the files separately.

We assume you have created the "kallisto" directory on your local host.  Since we have mapped the local directory */Users/username/nbdocker* on the host to */home/jovyan/work* inside the container running nbdocker, we assume the path */Users/username/nbdocker/kallisto* on your local host exists.  The fastq files will be downloaded to '/home/jovyan/work/kallisto' within the container running nbdocker.

**Click on the whale button** below to run this container

{nbdocker#0}

## 2. Perform quality assessment using a R/Bioconductor package

In [7]:
# Install the "ShortRead" Bioconductor package
source('http://bioconductor.org/biocLite.R')
biocLite("ShortRead")
library (ShortRead)

Bioconductor version 3.6 (BiocInstaller 1.28.0), ?biocLite for help
A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.6 (BiocInstaller 1.28.0), R 3.4.3 (2017-11-30).
Installing package(s) ‘ShortRead’
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'cluster', 'curl', 'devtools', 'evaluate', 'foreign', 'git2r',
  'glue', 'inline', 'IRdisplay', 'MASS', 'Matrix', 'mgcv', 'mvtnorm', 'nlme',
  'pbdZMQ', 'Rcpp', 'RcppArmadillo', 'RCurl', 'repr', 'robustbase', 'rrcov',
  'stringi', 'stringr', 'survival'


In [8]:
# list the fastq files already downloaded
list.files('/home/jovyan/work/kallisto')

[1] "output"               "SRR493366_1.fastq.gz" "SRR493366_2.fastq.gz"

In [9]:
# perform quality assessment on the downloaded fastq files
qaSummary <- qa('/home/jovyan/work/kallisto')
qaSummary

class: FastqQA(10)
QA elements (access with qa[["elt"]]):
  readCounts: data.frame(2 3)
  baseCalls: data.frame(2 5)
  readQualityScore: data.frame(1024 4)
  baseQuality: data.frame(190 3)
  alignQuality: data.frame(2 3)
  frequentSequences: data.frame(100 4)
  sequenceDistribution: data.frame(88 4)
  perCycle: list(2)
    baseCall: data.frame(1009 4)
    quality: data.frame(7324 5)
  perTile: list(2)
    readCounts: data.frame(0 4)
    medianReadQualityScore: data.frame(0 4)
  adapterContamination: data.frame(2 1)

In [10]:
# generate the assessment report
report(qaSummary, dest="/home/jovyan/work/kallisto")
getwd()

Warning message in dir.create(dest, recursive = TRUE):
“'/home/jovyan/work/kallisto' already exists”

[1] "/home/jovyan/work/kallisto/index.html"

[1] "/home/jovyan/work"

#### The assessment report is generated as an index.html file in the demo directory. You can click on the following link to view it in the browser.

[View FastQC Report](./kallisto/index.html)
